## Import packages

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.io as scio
import os
import mne
import pickle

from numba import jit, cuda

import pac
# import scipy.signal as ss

import Get_global_local_feature

## Get global feature using convolution

In [3]:
PAC_dist = np.ones(shape=[20,20])
PAC_dist[0:5,0:5] = PAC_dist[0:5,0:5]*2
PAC_dist[5:10,5:10] *= 3
PAC_dist[10:15,10:15]*= 0
PAC_dist[15:20,15:20]*= 10
Get_global_local_feature.conv_PAC_dist(PAC_dist)
# # Filtering
# width_filter = 5
# height_filter = 5
# kernel_filter = np.ones(shape=[width_filter,height_filter])/(width_filter*height_filter)
# PAC_feature = ss.convolve2d(PAC_dist,kernel_filter,mode='valid')
# PAC_feature.shape
# # Sampling
# height_index = np.linspace(0,PAC_feature.shape[0]-1,PAC_dist.shape[0]//height_filter).astype(np.int32)
# width_index = np.linspace(0,PAC_feature.shape[0]-1,PAC_dist.shape[1]//width_filter).astype(np.int32)
# h,w = np.meshgrid(height_index,width_index)
# PAC_feature[h,w]

array([[ 2.,  1.,  1.,  1.],
       [ 1.,  3.,  1.,  1.],
       [ 1.,  1.,  0.,  1.],
       [ 1.,  1.,  1., 10.]])

In [1]:
a = np.ones([3,3])
a[1,1] = 15
print(a)

u,v = np.meshgrid([1],[0,1,2])
print(u)
a[u,v]



NameError: name 'np' is not defined